In [1]:
import os
import joblib
import pickle
import random
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [2]:
def convert(train):
    path=[]
    for i in range(len(train)):
        path.append(train[i].replace("_img_","_mask_").replace("patch","patch_oh")[:-3]+"pkl")
    return path

In [3]:

def splitdataset(fold):
    train_dirs = []
    valid_dirs = []
    test_dirs = []
    img_path = '/data3/digestpath_img_patch'
    mask_path = '/data3/digestpath_mask_patch'
    dictionary=joblib.load("/home/houwentai/env/pyhwt/digestpath/dictionary.pkl")
    wsi = []
    wsi_label = []
    for level in os.listdir(img_path):
        for image in os.listdir(os.path.join(img_path,level)):
            wsi.append(os.path.join(img_path,level,image)) 
            if level=="normal":
                wsi_label.append(0)
            elif level =="low level":
                wsi_label.append(1)
            else:
                wsi_label.append(2)
    kf = StratifiedKFold(n_splits=fold,shuffle=True,random_state=0)
    train_dataset=[]
    val_dataset=[]
    test_dataset=[]
    num=1
    for train_index , test_index in kf.split(wsi,wsi_label):
        print("Fold"+str(num))
        new_train_index, val_index , _ , _ = train_test_split(train_index,train_index,test_size=0.2,random_state=0)
        train_img_patch_fold=[]
        val_img_patch_fold=[]
        test_img_patch_fold=[]
        train_label_patch_fold=[]
        val_label_patch_fold=[]
        test_label_patch_fold=[]
        filter_train_patch_name=[]
        train_gjb=[]
        train_djb=[]
        train_normal=[]
        for index in list(new_train_index):
            for patch_name in os.listdir(os.path.join(wsi[index])):
                if dictionary[os.path.join(wsi[index],patch_name)] != 0:
                    train_dirs.append(wsi[index])
                break
            # print(index)
            for patch_name in os.listdir(os.path.join(wsi[index])):
                if dictionary[os.path.join(wsi[index],patch_name)] == 0:
                    train_normal.append(os.path.join(wsi[index],patch_name))
                elif dictionary[os.path.join(wsi[index],patch_name)] == 1:
                    train_djb.append(os.path.join(wsi[index],patch_name))
                else:
                    train_gjb.append(os.path.join(wsi[index],patch_name))
        train_normal=random.sample(train_normal, max(len(train_gjb),len(train_djb)))
#         train_img_patch_fold=train_normal+train_djb+train_gjb
#         train_label_patch_fold=convert(train_normal)+convert(train_djb)+convert(train_gjb)
        train_img_patch_fold=train_djb+train_gjb
        train_label_patch_fold=convert(train_djb)+convert(train_gjb)
        print("Fold"+str(num)+"训练集统计：")
        print("正常图片数："+str(len(train_normal)))
        print("低级别图片数："+str(len(train_djb)))
        print("高级别图片数："+str(len(train_gjb)))
        val_gjb=[]
        val_djb=[]
        val_normal=[]
        for index in list(val_index):
            valid_dirs.append(wsi[index])
            # print(index)
            for patch_name in os.listdir(os.path.join(wsi[index])):
                if dictionary[os.path.join(wsi[index],patch_name)] == 0:
                    val_normal.append(os.path.join(wsi[index],patch_name))
                elif dictionary[os.path.join(wsi[index],patch_name)] == 1:
                    val_djb.append(os.path.join(wsi[index],patch_name))
                else:
                    val_gjb.append(os.path.join(wsi[index],patch_name))
        val_normal=random.sample(val_normal, max(len(val_gjb),len(val_djb)))
#         val_img_patch_fold=val_normal+val_djb+val_gjb
#         val_label_patch_fold=convert(val_normal)+convert(val_djb)+convert(val_gjb)
        val_img_patch_fold=val_djb+val_gjb
        val_label_patch_fold=convert(val_djb)+convert(val_gjb)
        print("Fold"+str(num)+"验证集统计：")
        print("正常图片数："+str(len(val_normal)))
        print("低级别图片数："+str(len(val_djb)))
        print("高级别图片数："+str(len(val_gjb)))

        test_gjb=0
        test_djb=0
        test_normal=0
        for index in list(test_index):
            print(wsi[index])
            test_dirs.append(wsi[index])
            for patch_name in os.listdir(os.path.join(wsi[index])):
                if dictionary[os.path.join(wsi[index],patch_name)] == 0:
                    test_normal+=1
                elif dictionary[os.path.join(wsi[index],patch_name)] == 1:
                    test_djb+=1
                else:
                    test_gjb+=1
                test_img_patch_fold.append(os.path.join(wsi[index],patch_name))
        test_label_patch_fold=convert(test_img_patch_fold)
        print("Fold"+str(num)+"测试集统计：")
        print("正常图片数："+str(test_normal))
        print("低级别图片数："+str(test_djb))
        print("高级别图片数："+str(test_gjb))
        num+=1
        print("-------------------------------------")
        break
    return train_dirs, valid_dirs, test_dirs

In [4]:
train_dirs, valid_dirs, test_dirs = splitdataset(4)

Fold1
Fold1训练集统计：
正常图片数：1372
低级别图片数：1372
高级别图片数：1119
Fold1验证集统计：
正常图片数：162
低级别图片数：162
高级别图片数：88
/data3/digestpath_img_patch/normal/2019-06-04 16_57_19.png
/data3/digestpath_img_patch/normal/2019-06-11 01_00_50.png
/data3/digestpath_img_patch/normal/2019-06-04 17_29_52.png
/data3/digestpath_img_patch/normal/2019-06-11 00_48_57.png
/data3/digestpath_img_patch/normal/2019-07804-1-1-1_2019-05-29 05_31_41.png
/data3/digestpath_img_patch/normal/2019-05147-1-1-1_2019-05-29 11_03_32.png
/data3/digestpath_img_patch/normal/2019-06-11 00_55_09.png
/data3/digestpath_img_patch/normal/2019-06-11 09_15_18.png
/data3/digestpath_img_patch/low level/1904076001_2019-10-28 10_25_20.png
/data3/digestpath_img_patch/low level/1905287001_2019-10-29 15_46_13.png
/data3/digestpath_img_patch/low level/1904100004_2019-10-28 11_31_27.png
/data3/digestpath_img_patch/low level/1905356002_2019-10-29 17_08_39.png
/data3/digestpath_img_patch/low level/1904076002_2019-10-28 10_30_08.png
/data3/digestpath_img_patch/high 

In [5]:
data = {
    "train_dirs":train_dirs,
    "valid_dirs":valid_dirs,
    "test_dirs":test_dirs,
}
with open("/data1/lzy2020/checkpoint/fold1.pkl", "wb") as f:
    pickle.dump(data, f)

In [6]:
print(len(train_dirs), len(valid_dirs), len(test_dirs))

12 11 18


In [7]:
train_dirs

['/data3/digestpath_img_patch/low level/1904100003_2019-10-28 11_27_21.png',
 '/data3/digestpath_img_patch/low level/1905157001_2019-10-28 18_22_53.png',
 '/data3/digestpath_img_patch/high level/2018_78758_1-1_2019-02-21 01_17_57.png',
 '/data3/digestpath_img_patch/high level/18_00991B_2019-05-07 21_27_54.png',
 '/data3/digestpath_img_patch/low level/1904100002_2019-10-28 11_21_41.png',
 '/data3/digestpath_img_patch/high level/18-01913B_2019-05-07 22_15_41.png',
 '/data3/digestpath_img_patch/low level/1904066003_2019-10-29 13_17_07.png',
 '/data3/digestpath_img_patch/high level/18-01454B_2019-05-07 21_53_08.png',
 '/data3/digestpath_img_patch/low level/1904099001_2019-10-28 11_14_27.png',
 '/data3/digestpath_img_patch/low level/1905307002_2019-10-29 16_38_40.png',
 '/data3/digestpath_img_patch/high level/2018_72821_1-1_2019-02-21 00_06_13.png',
 '/data3/digestpath_img_patch/high level/18-01454A_2019-05-07 21_47_50.png']